<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/111_%E6%95%B0%E5%80%A4%E3%81%AE%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

数値の処理
==========

数値型
------

### 整数型 ###

組み込みの `int` 型は、整数型である。公式配布の C による Python 実装（CPython）での `int` 型は、任意の長さの数をバイナリ形式で保存したもので、一般に `bignum` または多倍長整数として知られている。つまり、 CPython ではメモリの許す限り任意の桁数の整数値を表現できて、桁あふれ（オーバーフロー）が起こらない。

整数リテラルでは、アンダースコア `_` を使って数字をグループ化することで読みやすくできる。アンダースコアは、 2 つ以上つなげたり、数字の間以外の場所に使うことはできない。

接頭辞を付けて基数を表すことができる:

| 接頭辞 | 基数 | 使用可能文字 |
|:---|:---|:---|
| `0b` または `0B` | 2 （進数） | `0`, `1` |
| `0o` または `0O` | 8 （進数） | `0`, `1`, `2`, `3`, `4`, `5`, `6`, `7` |
| `0x` または `0X` | 16 （進数） | `0`, `1`, `2`, `3`, `4`, `5`, `6`, `7`, `8`, `9`, `A`, `B`, `C`, `D`, `E`, `F`（大文字小文字の区別なし） |

なお、`0` でない 10 進数の先頭には `0` を付けられない。

整数リテラルの例をいくつか示す:

``` python
7
100_000_000_000
0b100110111
0b_1110_0101
0o177
0xdeadbeef
```

コンストラクタは次のとおり。

``` python
int(number=0, /)
int(string, /, base=10)
```

数または文字列から生成された整数オブジェクトを返す。引数が指定されない場合は `0` を返す。

第 1 引数（位置専用引数）に文字列を指定する場合は、 `base` 引数を基数とした整数を返す。 `base` 引数を省略した時は 10 進数表現として解釈される。文字列は先頭に `+` または `-` を伴ったり（符号と数値の間に空白文字を入れてはいけない）、数値の先頭に任意の数のゼロを付けたり、前後に空白文字を付けたり、各桁を 1 つだけのアンダースコアで区切ったりしてもよい。

基数を `0` に指定した場合、文字列は整数リテラルと同じように解釈される。

In [ ]:
assert int() == 0
assert int(123.45) == 123  # 小数点以下切り捨て
assert int('123') == 123
assert int('   -12_345\n') == -12345
assert int('01110011', base=2) == 115
assert int('FACE', 16) == int('0xface', 0) == 64206

なお、Python 3.10.7（および、同日公開されたバージョン 3.9.14, 3.8.14）以降では、<font color="red">整数と文字列の変換が `sys.int_info.default_max_str_digits` 桁（デフォルトで 4300 桁）までに制限されるようになった</font>（制限違反の場合は `ValueError` を送出する）。ただし、この制限は基数が 2 のべき乗の場合（2進数、8進数、16進数などの場合）には適用されないので、 `int(string, base)` の呼び出しは `base` が `2`、`4`、`8`、`16` または `32` の場合は制限がない。

この変更は、基数が 2 のべき乗でない限り、多倍長整数と文字列の間の変換アルゴリズムの計算量が $O(n^2)$ であるため、DoS 攻撃（Denial of Service attack; サービス拒否攻撃）に悪用される可能性があったことによる。

あくまでも整数・文字列変換が制限されるだけで、`int` 型自体は 4300 桁より大きい値を保持できる。

In [ ]:
import sys
print(f"{sys.int_info.default_max_str_digits=}")
i = int('1' * 4301, base=2)  # 2 進数としての変換はエラーにならない
assert int('1' * 4300, base=2) < i
try:
    i = int('1' * 4301)
except Exception as e:
    print(f"{type(e).__name__}: {e}")
try:
    s = str(10**4300)  # 4301 桁の整数自体はエラーにならない（変換でエラーが発生）
except Exception as e:
    print(f"{type(e).__name__}: {e}")

sys.int_info.default_max_str_digits=4300
ValueError: Exceeds the limit (4300) for integer string conversion: value has 4301 digits; use sys.set_int_max_str_digits() to increase the limit
ValueError: Exceeds the limit (4300) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit


`int` 型は以下の追加のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `as_integer_ratio()` | 整数の値を有理数として表現する（分母が常に `1`） | `tuple` |
| `is_integer()` | 常に `True` を返す（Python 3.12 で追加）。`float.is_integer()` に対するダックタイピングの互換性のために存在している | `True` |

In [ ]:
assert (123).as_integer_ratio() == (123, 1)

`bool` は `int` のサブクラスである。このクラスのインスタンスはシングルトンの `False` と `True` のみであり、それぞれ `0` と `1` に等価である。しかし、その等価性に頼ることは推奨されない。`int()` を使って明示的に整数値に変換すること。

ちなみに、`bool` のコンストラクタは、引数のオブジェクトに対する真理値の判定を返す。

In [ ]:
assert bool(0) is bool() is False == 0  # 引数なしのコンストラクタは False を返す
assert bool(object) is True == 1

組み込み関数 `hex()` は、引数に整数を受け付け、整数を、先頭に `'0x'` が付いた小文字の 16 進文字列に変換する。

In [ ]:
hex(255)

'0xff'

### 浮動小数点数型 ###

組み込みの `float` 型は、浮動小数点数型（実数型）である。固定長（64ビット倍精度）であり、 `- sys.float_info.max` から `sys.float_info.max` までの範囲の値を表現できる。

In [ ]:
import sys
print(f"{sys.float_info.max=}")

sys.float_info.max=1.7976931348623157e+308


浮動小数点リテラルは、`3.14` のような固定小数点表記と、`3.14e-10` のような指数表記が可能で、常に `10` 進数で解釈される。整数リテラルと同様に、桁のグループ化にはアンダースコア `_` がサポートされている。小数点の前だけ、または小数点の後ろだけを省略した表記 `.001`, `10.` も可能である。

コンストラクタは次のとおり。

``` python
float(number=0.0, /)
float(string, /)
```

数または文字列から生成された浮動小数点数を返す。

引数が文字列の場合、10進数でなければならない。先頭に符号または空白を含んでいてもよいが、無限大や非数以外の数値を取得するには、浮動小数点リテラルの表記規則に従う必要がある。

In [ ]:
assert float() == 0.0
assert float('+1.23') == 1.23
assert float('   -12345\n') == -12345.0
assert float('1e-003') == 0.001
assert float('+1E6') == 1000000.0
assert float('.12') == 0.12  # 整数部に 0 が補完されて変換される
assert float('100') == 100.  # 整数の文字列も float 型の数値に変換される

`int` 型と `float` 型との間では比較が可能である。

In [ ]:
assert 1 == 1.0
assert 1 < 1.01

`float` 型は以下の追加のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `as_integer_ratio()` | 浮動小数点数インスタンスの値を有理数として表現する | `tuple` |
| `is_integer()` | 浮動小数点数インスタンスが有限の整数値なら `True` を、そうでなければ `False` を返す | `bool` |
| `hex()` | 浮動小数点数インスタンスの 16 進文字列表現を返す。有限の浮動小数点数に対し、この表現は常に `'0x'` で始まり `'p'` と指数が続く | `str` |
| `fromhex(s)` | クラスメソッド。`hex()` の逆の変換を行う | `float` |

In [ ]:
assert (2.75).as_integer_ratio() == (11, 4)  # 11/4
assert (-2.0).is_integer()
x = (2.75).hex()
print(f"{x=}")
assert float.fromhex(x) == 2.75

x='0x1.6000000000000p+1'


### 算術演算の型規則 ###

| 演算 | `int` 同士 | `float` 同士 | `int` と `float` | 備考 |
|:--:|:--:|:--:|:--:|:---|
| `x + y` （足し算） | `int` | `float` | `float` | |
| `x - y` （引き算） | `int` | `float` | `float` | |
| `x * y` （掛け算） | `int` | `float` | `float` | |
| `x / y` （割り算） | `float` | `float` | `float` | |
| `x // y` （整数除算） | `int` | `float` | `float` | `x` と `y` の商を切り下げたもの |
| `x % y` （剰余演算） | `int` | `float` | `float` | |
| `x ** y` （べき乗） | `int` | `float` | `float` | |

整理すると

  1. 割り算（`/`）は常に `float` 型を返す。
  2. `float` 型を含む計算は常に `float` 型を返す。
  3. `int` 型同士の計算は `int` 型を返すが、割り算（`/`）は除く。

### 複素数型 ###

組み込みの `complex` 型は、複素数型である。複素数は浮動小数点数と虚数のペアとして表され、その範囲に対して浮動小数点数の制限を受ける。

虚数リテラルは、末尾に `j` または `J` を付ける。虚数リテラルは、実部が `0.0` の複素数を生成する。 `0.0` 以外の実部を持つ複素数を作成するには、それに浮動小数点数を追加する（例: `3+4j`）。

コンストラクタは次のとおり。

``` python
complex(number=0, /)
complex(string, /)
complex(real=0, imag=0)
```

In [ ]:
assert complex() == 0
assert complex('+1.23') == 1.23+0j
assert complex('\t( -1.23+4.5J )\n') == complex(-1.23, 4.5) == -1.23+4.5j
assert complex(imag=-4.5) == -4.5j

複素数インスタンスは、`real` 属性、`imag` 属性、 `conjugate()` メソッドを持つ:

In [ ]:
assert (1+3j).real == 1.0  # 実部
assert (1+3j).imag == 3.0  # 虚部
assert (1+3j).conjugate() == (1-3j)  # 複素共役

### 抽象基底クラス ###

標準ライブラリの `numbers` は、数の抽象基底クラスの階層を定義している。

| ABC | 意味 | 継承しているクラス | 受け付ける演算・関数 | 抽象メソッド・抽象プロパティ |
|:---|:---|:---|:---|:---|
| `Number` | 数の階層のルート | | | | |
| `Complex` | この型のサブクラスは複素数を表す | `Number` | `+`, `-`, `*`, `/`, `**`, `complex()`, `bool()`, `abs()`, `==`, `!=` | `conjugate()`, `real`, `imag` |
| `Real` | 実数を扱う演算を追加したもの | `Complex` | `+`, `-`, `*`, `/`, `**`, `complex()`, `bool()`, `abs()`, `==`, `!=`<br />`float()`, `round()`, `divmod()`, `//`, `%`, `<`, `<=`, `>`, `>=` | `conjugate()`, `real`, `imag` |
| `Rational` | 有理数を扱うプロパティを加えたもの | `Real` | `Real` と同じ | `conjugate()`, `real`, `imag`, `numerator`, `denominator` |
| `Integral` | 整数への変換を加えたもの | `Rational` |  `+`, `-`, `*`, `/`, `**`, `complex()`, `bool()`, `abs()`, `==`, `!=`<br />`int()`, `float()`, `round()`, `divmod()`, `//`, `%`, `<`, `<=`, `>`, `>=` | `conjugate()`, `real`, `imag`, `numerator`, `denominator` |

引数 `x` が、種類は何であれ、数であるということだけチェックしたい場合、`isinstance(x, numbers.Number)` が使える。

In [ ]:
import numbers
assert issubclass(numbers.Complex, numbers.Number)
assert isinstance(complex(), numbers.Complex)
assert isinstance(float(), numbers.Real)
assert isinstance(int(), numbers.Integral)
assert isinstance(bool(), numbers.Integral)

計算機イプシロン
----------------

IEEE 754 浮動小数点規格により、コンピューターで扱われる浮動小数点数は、`1.xxx...e指数` という形の指数表記とされ、指数を格納する指数部と、`xxx...` を格納する仮数部に分かれている。このため仮数部の精度（ビット数）は、1 より大きい最小の数の限界となる。この限界（つまり 1 より大きい最小の数と 1 との差）のことを**計算機イプシロン**（machine epsilon）といい、 $\epsilon$ で表す。Python の `float` 型が扱う 64ビット倍精度では、$\epsilon=2^{{1-53}}$ となる。この数値は、`sys.float_info.epsilon` で確認できる。

In [ ]:
import sys
sys.float_info.epsilon

2.220446049250313e-16

コンピューターで扱う数値は 2 進数であり、分数が

$$
\dfrac{a}{2^n}
$$

という形になる値でない限り、無限小数になる。2 進小数への変換に伴う誤差と計算機イプシロンの存在により、コンピューターによる小数計算の結果は、 10 進法での計算結果とは異なることが多い（公式チュートリアルのトピック「[浮動小数点演算、その問題と制限](https://docs.python.org/ja/3/tutorial/floatingpoint.html#tut-fp-issues)」参照）。

In [ ]:
assert 0.1 + 0.1 + 0.1 != 0.3

無限大
------

浮動小数点数の演算が桁あふれを起こして、仮数部が 0 で、指数部が最大値になった状態は、符号が `-` でないなら**（正の）無限大**、符号が `-` なら**負の無限大**とされる。

コンストラクタ `float()` の引数に文字列 `'inf'` または `'infinity'` を指定すると、正の無限大を生成する。小文字と大文字の区別はなく、小文字と大文字が混ざっていても問題ない。また、文字列 `'-inf'` または `'-infinity'` を指定すると、負の無限大を生成する。小文字と大文字の区別がないことは同様。`float('-inf')` と `- float('inf')` は等価である。

正の無限大も負の無限大も、浮動小数点数の特殊な値とされ、比較 `==`、`!=`、`<`、`>`、`<=`、`>=` が可能である。浮動小数点数との比較では、正の無限大はどの浮動小数点数よりも大きく、負の無限大はどの浮動小数点数よりも小さい。一方、整数との比較では、Python の整数表現に上限がないのであるが、正の無限大はどの整数よりも大きいとされ、また、負の無限大はどの整数よりも小さいとされる。

無限大から整数への変換は `OverflowError` 例外を送出する。また、浮動小数点数の最大値を超える整数から浮動小数点数への変換も `OverflowError` 例外を送出する。

In [ ]:
import sys

inf = float("inf")
assert str(inf) == "inf"
assert isinstance(inf, float)  # 無限大は float 型の値である
assert inf > sys.float_info.max
assert inf == sys.float_info.max * 2  # 意図的に桁あふれを起こさせる

large_int = int(sys.float_info.max) * 2
assert inf > large_int > sys.float_info.max  # 整数との比較でも、無限大は常に大きい
#int(inf) # OverflowError
#float(large_int) # OverflowError

ひとたび浮動小数点数が正の無限大となると、その値と、正の無限大でも負の無限大でもない値との和・差・積・商は正の無限大となる。負の無限大についても同様である。また、べき乗については、無限大の 0 乗は 1 で、1 の 無限大乗は 1 で、0 の 無限大乗は 0 で、それ以外は無限大となる。

In [ ]:
inf = float("inf")
assert inf + 1.0 == inf - 1.0 == inf * 2 == inf / 2 == inf ** 2 == 2 ** inf == inf
assert inf ** 0 == 1 ** inf == 1

正の無限大（または負の無限大）同士の和・積・べき乗も定義され、いずれも正の無限大（または負の無限大）となる。

In [ ]:
inf = float("inf")
assert inf + inf == inf * inf == inf ** inf == inf

標準ライブラリの `math` モジュールが提供する定数 `math.inf` は正の無限大を表す。また、`math.isinf(x)` 関数は `x` が正ないし負の無限大ならば `True` を返し、それ以外の場合に `False` を返す。

In [ ]:
import math
import sys
assert math.inf == float("inf")
assert math.isinf(sys.float_info.max * 2)

非数
----

**非数**は、NaN（**N**ot **a** **N**umber）と略し、主に浮動小数点演算において、不正な対象を与えられたために生じた結果を表す値またはシンボルとして使われる。IEEE 754 浮動小数点規格では、浮動小数点の指数部が最大値で、仮数部が 0 でない状態が NaN とされる。

Python では、正の無限大（または負の無限大）同士の差・商は、NaN となる。

コンストラクタ `float()` の引数に文字列 `'nan'` を指定すると、NaN を生成する。小文字と大文字の区別はなく、小文字と大文字が混ざっていても問題ない。

NaN は、浮動小数点数の特殊な値とされるが、いかなる数値とも比較 `==`、`<`、`>`、`<=`、`>=` が `False` となる。NaN 同士でも同様で、`float('nan') == float('nan')` の評価結果は `False` となる。一方、NaN そのものは、真偽値が要求される場面で `True` と評価される。

NaN を含む `+`, `-`, `*`, `/`, `**` などの演算結果はすべて NaN となる。

In [ ]:
nan = float("NaN")
print(f"{nan = }")
inf = float('inf')
print(f"{inf - inf = }")  # 無限大同士の差は NaN となる
assert isinstance(nan, float)  # NaN は float 型の値である
assert nan != nan
assert bool(nan)

nan = nan
inf - inf = nan


`math` モジュールが提供する定数 `math.nan` は NaN を表す。また、`math.isnan(x)` 関数は `x` が NaN ならば `True` を返し、それ以外の場合に `False` を返す。

In [ ]:
import math
assert math.isnan(math.nan + 1)  # NaN を含む演算の結果は NaN

数値演算
--------

### 組み込み関数 ###

以下は、数値演算を行う組み込み関数であり、引数の `x` や `y`、`z` は 浮動小数点数または整数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:--:|
| `abs(x)` | `x` の絶対値を返す | 数値 |
| `divmod(x, y)` | `(x // y, x % y)` という 2 要素タプルを返す | `tuple` |
| `pow(x, y)` | `x ** y` と等価である | 数値 |
| `pow(x, y, z)` | `x` の `y` 乗を `z` で割ったときの剰余を返す。`pow(x, y) % z` より効率よく計算される | 数値 |
| `max(iterable, *, default, key=None)` | `iterable` の最大の要素が返される。`key` 引数は引数を 1 つ取る順序関数（`list.sort()` のもののように）を指定できる。`default` 引数は<br /> `iterable` が空の場合に返すオブジェクトを指定する。`iterable` が空で `default` が与えられていない場合 `ValueError` 例外が送出される | 数値 |
| `max(arg1, arg2, *args, key=None)` | 2 つ以上のキーワード無しの位置引数が与えられた場合、その位置引数の中で最大のものが返される。`key` は上と同じ | 数値 |
| `min(iterable, *, default, key=None)` | `iterable` の最小の要素が返される。`default` と `key` は上と同じ | 数値 |
| `min(arg1, arg2, *args, key=None)` | 2 つ以上のキーワード無しの位置引数が与えられた場合、その位置引数の中で最小のものが返される。`key` は上と同じ | 数値 |
| `sum(iterable, /, start=0)` | `start` と `iterable` の総和を足した値を返す | 数値 |
| `round(x[, n])` | 丸め処理を行う。具体的には `x.__round__(n)` を呼び出す。`n` が省略された場合は `x.__round__(0)` を呼び出す | 数値 |

`x.__round__(n)` メソッドは、$10^{-n}$ の桁の精度に丸めた値を返す。

  * 引数 `n` が `2` の場合、$10^{-2}\;(=0.01)$ の桁の精度、すなわち小数点以下第 2 位に丸める。
  * 引数 `n` が `1` の場合、$10^{-1}\;(=0.1)$ の桁の精度、すなわち小数点以下第 1 位に丸める。
  * 引数 `n` が `0` の場合は、元の値に最も近い整数を返す。

丸め方は「$10^{-n}$ の倍数の中で最も近いものに丸める」というもの。たとえば、`1.234` を小数点以下第 1 位に丸める場合、`0.1` の倍数の中で `1.234` に一番近い `1.2` に丸める。`1.267` を小数点以下第 1 位に丸める場合、`0.1` の倍数の中で `1.267` に一番近い `1.3` に丸める。1 つ下の桁に対して四捨五入する処理に相当する。

**「最も近いもの」が 2 つある場合は、どちらに丸めるのか予測しづらい**。一応、「偶数を選ぶ」というルールはあって、これは「偶数丸め」や「銀行家の丸め」と呼ばれる。たとえば `1.25` を小数点以下第 1 位に丸める場合、`0.1` の倍数の中では `1.2` も `1.3` も `1.25` に同等に「最も近いもの」であるが、`1.2` を選ぶ。つねに五入するのではなく、五入と五捨を偶奇性で決めてバランスをとっているのである。しかしながら、ほとんどの小数は浮動小数点数では正確に表せないため、人間が考えるルールどおりにはならない。たとえば、「偶数丸め」なら、`1.15` を小数点以下第 1 位に丸める場合に `1.2` になるはずだが、`round(1.15, 1)` は `1.1` を返す。

なお、引数 `n` は負の整数とすることもできる。`n` が `-1` の場合は、1 の位で四捨五入する。`n` が `-2` の場合は、10 の位で四捨五入する。

In [ ]:
assert abs(-123) == 123
assert abs(3+4j) == 5.0  # 複素数の絶対値: 3**2+4**2 の非負の平方根
assert divmod(123, 45) == (2, 33)
assert pow(2, 4) == 16
assert pow(123, 45, 6) == 3
assert max(24, 12, 6) == 24
assert min(24, 12, 6) == 6
assert sum([1, 2, 3], 100) == 106
assert round(0.1234, 2) == 0.12
assert round(0.1267, 2) == 0.13
assert round(123.45, -2) == 100.0
assert round(167.89, -2) == 200.0

### 数学関数 ###

以下は、`math` モジュールが提供する数学関数であり、引数の `x` や `y` は 浮動小数点数または整数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `math.prod(iterable, *, start=1)` | `start` と `iterable` の総乗を掛けた値を返す | 数値 |
| `math.ceil(x)` | `x` の天井（`x` 以上の最小の整数）を返す | `int` |
| `math.floor(x)` | `x` の床（`x` 以下の最大の整数）を返す | `int` |
| `math.trunc(x)` | `x` の小数点以下を切り捨てる | `int` |
| `math.fabs(x)` | `x` の絶対値を返す | 数値 |
| `math.gcd(*integers)` | 整数引数の最大公約数を返す | `int` |
| `math.lcm(*integers)` | 整数引数の最小公倍数を返す | `int` |
| `math.log(x[, base])` | 引数が 1 つの場合、`x` の（e を底とする）自然対数を返す。引数が 2 つの場合、`base` を底とする `x` の対数を返す | `float` |
| `math.log10(x)` | 10 を底とする `x` の対数（常用対数）を返す。`log(x, 10)` よりも高精度 | `float` |
| `math.log2(x)` | 2 を底とする `x` の対数を返す。`log(x, 2)` よりも高精度 | `float` |
| `math.pow(x, y)` | `x` の `y` 乗を返す。組み込みの `**` 演算子と違って、この関数は両方の引数を `float` 型に変換する | `float` |
| `math.sqrt(x)` | `x` の平方根を返す。`math.pow(x, 1/2)` と同等 | `float` |
| `math.radians(x)` | 角 `x` を度からラジアンに変換する | `float` |
| `math.degrees(x)` | 角 `x` をラジアンから度に変換する | `float` |
| `math.cos(x)` | `x` ラジアンの余弦（Cosine）を返す | `float` |
| `math.sin(x)` | `x` ラジアンの正弦（Sine）を返す | `float` |
| `math.tan(x)` | `x` ラジアンの正接（Tangent）を返す | `float` |
| `math.acos(x)` | `x` の逆余弦をラジアンで返す。結果は 0 と $\pi$ の間となる | `float` |
| `math.asin(x)` | `x` の逆正弦をラジアンで返す。結果は $-\pi/2$ と $\pi/2$ の間となる | `float` |
| `math.atan(x)` | `x` の逆正接をラジアンで返す。結果は $-\pi/2$ と $\pi/2$ の間となる | `float` |
| `math.cosh(x)` | `x` の双曲線余弦を返す | `float` |
| `math.sinh(x)` | `x` の双曲線正弦を返す | `float` |
| `math.tanh(x)` | `x` の双曲線正接を返す | `float` |
| `math.acosh(x)` | `x` の逆双曲線余弦を返す | `float` |
| `math.asinh(x)` | `x` の逆双曲線正弦を返す | `float` |
| `math.atanh(x)` | `x` の逆双曲線正接を返す | `float` |

なお、`math` モジュールは以下の数学定数も提供している:

  * `math.e`: ネイピア数（自然対数の底）
  * `math.pi`: $\pi$（円周率）

In [ ]:
import math
assert math.prod(range(1, 6), start=10) == 1200
assert math.ceil(1.7) == 2  # 1.7 以上の最小の整数
assert math.floor(1.7) == 1  # 1.7 以下の最大の整数
assert math.trunc(-1.7) == math.ceil(-1.7) == -1
assert math.fabs(-1.7) == 1.7
assert math.gcd(12, 18) == 6  # 12 = 2·3·4·6 と 18 = 2·3·6·9 の最大公約数
assert math.lcm(12, 18) == 36  # (12, 24, 36, ...) と (18, 36, 54, ...) の最小公倍数
assert math.log10(100) == 2.0  # 10 ** 2.0 = 100
assert math.pow(2, 4) == 16.0
assert round(math.sqrt(2), 3) == 1.414
assert math.degrees(math.pi) == 180.0
assert math.cos(math.pi) == -1.0
assert math.sin(math.radians(90)) == 1.0
assert round(math.tan(math.radians(45))) == 1
assert math.acos(-1.0) == math.pi  # acos() は cos() の逆関数
assert math.asin(1.0) == math.radians(90)  # asin() は sin() の逆関数
assert math.atan(1.0) == math.radians(45)  # atan() は tan() の逆関数
x = 2
y1 = math.cosh(x)
y2 = (math.pow(math.e, x) + math.pow(math.e, -x)) / 2
assert round(y1, 10) == round(y2, 10)
y1 = math.sinh(x)
y2 = (math.pow(math.e, x) - math.pow(math.e, -x)) / 2
assert round(y1, 10) == round(y2, 10)
y1 = math.tanh(x)
y2 = math.sinh(x) / math.cosh(x)
assert round(y1, 10) == round(y2, 10)

十進小数演算
------------

### 十進数型 ###

利息計算、消費税計算、給与計算の端数処理のように 10 進法での小数の計算や丸めが必要な場合は、標準ライブラリの `decimal` モジュールが提供するクラス `decimal.Decimal` （十進数型）を使う。

`decimal.Decimal` オブジェクトの基本的な性質は、次のとおり。

  * 数値型と同じように比較や演算が行える。
  * `str()` 関数に渡すと、与えられた数値を表す文字列が得られる。
  * イミュータブルで、かつ、ハッシュ可能である。
  * 有効桁数の表記が取り入れられ、末尾のゼロは有効数字を示すために残される。

``` python
decimal.Decimal(value='0', context=None)
```

`value` として 10 進小数を表現する文字列を渡してこのコンストラクタを呼び出すと、`value` の値を持つ `decimal.Decimal` のインスタンスが作成される。無限大や非数を表す文字列 `'Infinity'`, `'-Infinity'`, `'NaN'` を渡すこともできる。浮動小数点数を渡すこともできるが、そうすると浮動小数点数自体に含まれる誤差を持ち込むことになるので注意すること。

In [ ]:
from decimal import Decimal
from numbers import Number

d = Decimal(str(0.1))
assert isinstance(d, Number)
print(f"{str(d)=}")
assert d + d + d == Decimal("0.3")  # 10進法での計算ができている
f = Decimal(0.1)
print(f"{str(f)=}")
assert f + f + f != Decimal("0.3")  # 浮動小数点数を使うと浮動小数点数に含まれる誤差による影響が出る

assert Decimal("1.30") + Decimal("1.20") == Decimal("2.50")  # 有効桁数 3 桁
assert Decimal("1.30") * Decimal("1.20") == Decimal("1.5600")  # 乗算では被演算子すべての桁数を使う方式をとる
assert Decimal("1.30") == Decimal("1.3000")
assert {d: "hoge"}[d] == "hoge"  # 辞書のキーに使える
Decimal("Infinity") + Decimal("NaN")

str(d)='0.1'
str(f)='0.1000000000000000055511151231257827021181583404541015625'


Decimal('NaN')

`decimal.Decimal` は、特別な演算を行うメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `exp(context=None)` | `value` での自然指数関数 `e ** value` の値を返す | `Decimal` |
| `ln(context=None)` | `value` の自然対数（底 e の対数）を返す | `Decimal` |
| `log10(context=None)` | `value` の常用対数（底 10 の対数）を返す | `Decimal` |
| `sqrt(context=None)` | `value` の平方根を返す | `Decimal` |
| `quantize(exp, rounding=None,`<br />` context=None)` | `value` を丸めた値を返す。`exp` で、求めたい桁数と同じ桁数の数値を `Decimal('0.1')` や `Decimal('0.01')` のように指定する。`rounding` で、<br />丸め方法を定数で指定できる | `Decimal` |

丸め方法を指定する定数は、次のとおり。

| 定数 | 意味 |
|:---|:---|
| `decimal.ROUND_HALF_EVEN` | 近い方に、引き分けは偶数整数方向に向けて丸める（デフォルト） |
| `decimal.ROUND_UP` | 切り上げ |
| `decimal.ROUND_DOWN` | 切り捨て |
| `decimal.ROUND_CEILING` | 正の無限大方向への丸め |
| `decimal.ROUND_FLOOR`| 負の無限大方向への丸め |
| `decimal.ROUND_HALF_UP` | 四捨五入 |
| `decimal.ROUND_HALF_DOWN` | 五捨六入 |
| `decimal.ROUND_05UP` | ゼロ方向に丸めた後の最後の桁が 0 または 5 ならばゼロから遠い方向に、そうでなければゼロ方向に丸める |

In [ ]:
from decimal import *

print(f"{Decimal('2').exp()=}")  # e^2
print(f"{Decimal('2').ln()=}")  # 2の自然対数
print(f"{Decimal('2').log10()=}")  # 2の常用対数
print(f"{Decimal('2').sqrt()=}")  # 2の平方根

exp = Decimal("0.1")  # 小数点第1位
assert Decimal("1.15").quantize(exp) == Decimal("1.2")  # 偶数丸め
assert Decimal("1.25").quantize(exp) == Decimal("1.2")  # 偶数丸め
assert Decimal("1.25").quantize(exp, ROUND_HALF_UP) == Decimal("1.3")  # 四捨五入
assert Decimal("1.25").quantize(exp, ROUND_HALF_DOWN) == Decimal("1.2")  # 五捨六入
assert Decimal("1.04").quantize(exp, ROUND_UP) == Decimal("1.1")  # 切り上げ
assert Decimal("1.06").quantize(exp, ROUND_DOWN) == Decimal("1.0")  # 切り捨て

Decimal('2').exp()=Decimal('7.389056098930650227230427461')
Decimal('2').ln()=Decimal('0.6931471805599453094172321215')
Decimal('2').log10()=Decimal('0.3010299956639811952137388947')
Decimal('2').sqrt()=Decimal('1.414213562373095048801688724')


### 算術コンテキスト ###

`decimal.Decimal` のコンストラクタやメソッドの `context` オプションには、`decimal.Context` オブジェクトを指定できる。`decimal.Context` オブジェクトは、 `decimal.Decimal` の算術演算における環境設定を管理するためのオブジェクトであり、次の属性を持つ。

| 属性 | 意味 |
|:---|:---|
| `prec` | 計算精度を表す整数。デフォルトでは 28 桁 |
| `rounding` | 丸め方法を定数で指定する |
| `traps` | この属性はシグナルをキーとする辞書を参照していて、値に真偽値を指定することで送出するエラーを制御できる |

`context` を指定しない場合は、現在有効な `decimal.Context` オブジェクトを指定したのと同様である。`decimal` モジュールをロードすると、自動的に `decimal.Context` オブジェクトが作成されて有効になっている。`decimal.getcontext()` 関数を使うと現在有効な `decimal.Context` オブジェクトにアクセスでき、設定を変更できる。

In [ ]:
from decimal import Decimal, getcontext

x = Decimal("10")
y = Decimal("7")
print(f"{x / y = }")
getcontext().prec = 3
print(f"{x / y = }")
getcontext().traps  # traps の設定を確認

x / y = Decimal('1.428571428571428571428571429')
x / y = Decimal('1.43')


{<class 'decimal.InvalidOperation'>:True, <class 'decimal.FloatOperation'>:False, <class 'decimal.DivisionByZero'>:True, <class 'decimal.Overflow'>:True, <class 'decimal.Underflow'>:False, <class 'decimal.Subnormal'>:False, <class 'decimal.Inexact'>:False, <class 'decimal.Rounded'>:False, <class 'decimal.Clamped'>:False}

`decimal.Context` オブジェクトの `traps` 属性のキー（シグナル）に対する値が `True` になっているものは、そのシグナルを例外として送出する。`decimal.FloatOperation` シグナルを `True` に指定すると、`float` 型が混在する場面でエラーが発生するようになる。これにより、浮動小数点数に含まれる誤差が持ち込まれることを防ぐことができる。

In [ ]:
from decimal import Decimal, getcontext, FloatOperation

getcontext().traps[FloatOperation] = True
try:
    d = Decimal(0.1)  # float を渡すとエラーが発生する
except FloatOperation:
    print("コンストラクタでFloatOperationが発生")
try:
    Decimal("0.3") < 1.5  # float と比較するとエラーが発生する
except FloatOperation:
    print("比較でFloatOperationが発生")

コンストラクタでFloatOperationが発生
比較でFloatOperationが発生


分数計算
--------

分数（有理数）の計算が必要な場合は、標準ライブラリの `fractions` モジュールが提供するクラス `fractions.Fraction` を使う。

コンストラクタ `fractions.Fraction()` の引数は、いくつかのタイプがある。

In [ ]:
from fractions import Fraction

assert Fraction(10, -8) == Fraction(-5, 4)  # 分子と分母を整数で指定
assert Fraction() == Fraction(0, 1)  # 分子のデフォルトは0、分母のデフォルトは1
assert Fraction(Fraction(1, 7), 5) == Fraction(1, 35)  # 分子を Fraction で指定
assert Fraction(Fraction(1, 7), Fraction(2, 3)) == Fraction(3, 14)  # 分母も Fraction で指定
assert Fraction('-35/4') == Fraction(-35, 4)  # 文字列で指定
assert Fraction('1.47') ==  Fraction(147, 100)
assert Fraction(1.47) == Fraction(6620291452234629, 4503599627370496)  # 浮動小数点数で指定

# 十進数型にも対応している
from decimal import Decimal
assert Fraction(Decimal('1.47')) ==  Fraction(147, 100)  # 十進数型で指定

`fractions.Fraction()` は `numbers.Rational` のサブクラスである。

In [ ]:
from fractions import Fraction
from numbers import Rational

assert issubclass(Fraction, Rational)
assert isinstance(Fraction(1, 2), Rational)

分数の計算は、算術演算子を使って `fractions.Fraction` オブジェクト同士の計算ができる。

In [ ]:
from fractions import Fraction

assert Fraction(1, 2) + Fraction(1, 3) == Fraction(5, 6)
assert Fraction(1, 2) - Fraction(1, 3) == Fraction(1, 6)
assert Fraction(2, 3) * Fraction(3, 4) == Fraction(1, 2)
assert Fraction(2, 3) / Fraction(3, 4) == Fraction(8, 9)

比較演算子による比較も可能。

In [ ]:
from fractions import Fraction
assert Fraction(7, 13) > Fraction(8, 15)

文字列、浮動小数点数、整数への変換は、それぞれ `str()`, `float()`, `int()` を使う。また、`fractions.Fraction` と `float` との演算結果は自動的に `float` に変換される。

In [ ]:
from fractions import Fraction

assert str(Fraction(2, 3)) == "2/3"
assert float(Fraction(2, 3)) == 0.6666666666666666
assert int(Fraction(4, 3)) == 1  # 小数点以下切り捨て
assert Fraction(2, 3) * 1.0 == 0.6666666666666666

`fractions.Fraction` のプロパティ（読み出し専用）:

| プロパティ | 意味 |
|:---|:---|
| `numerator` | 有理数を既約分数で表したときの分子 |
| `denominator` | 有理数を既約分数で表したときの分母 |

`fractions.Fraction` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `limit_denominator(max_denominator=1000000)` | 分母の上限を `max_denominator` に制限した有理数近似を返す | `Fraction` |

In [ ]:
from fractions import Fraction

pi = Fraction(3.141592653589793)  # 円周率
assert pi.limit_denominator(10) == Fraction("22/7")
assert pi.limit_denominator(1000) == Fraction("355/113")
assert Fraction(0.3333333333333333).limit_denominator() == Fraction("1/3")

統計計算
--------

**算術平均（相加平均）**:

　母平均（$N$ は母集団のデータ数）

$$
\mu = \dfrac{x_1 + x_2 + \cdots + x_N}{N}
$$

　標本平均（$n$ は標本のデータ数）

$$
\overline{x} = \dfrac{x_1 + x_2 + \cdots + x_n}{n}
$$

　算術平均は分布の中心を測る一般的な数値である。

**幾何平均（相乗平均）**:

$$
\overline{x}_G = \sqrt[n]{x_1 \times x_2 \times \cdots \times x_n}
$$

　幾何平均は比率や割合で変化するものに対してその平均を求めるときに使う。ただし、データに負の値を含めることはできない。

**調和平均**:

$$
\overline{x}_H = \dfrac{n}{\dfrac{1}{x_1} + \dfrac{1}{x_2} + \cdots + \dfrac{1}{x_n}}
$$

　調和平均は速度のような比率の平均を求めるときに使う。

**最頻値**:

　最も多く出現している値。

**n 分位数**:

　データを小さい順に並び替えたときに、データの数で $n$ 等分した時の $n-1$ 個の区切り値のこと。  
　四分位数では小さい順に**第一四分位数（25 パーセンタイル）**、**中央値（50パーセンタイル）**、**第三四分位数（75パーセンタイル）**と呼ぶ。  
　データが偶数個の場合は、中央に最も近い 2 つの値の平均値を中央値とする。  
　また、第三四分位数から第一四分位数を引いた値を**四分位範囲**と呼ぶ。  

**母分散**:

$$
\sigma^2 = \dfrac{(x_1 - \mu)^2 + (x_2 - \mu)^2 + \cdots + (x_N - \mu)^2}{N}
$$

　分散は平均からの平均二乗距離である。小さいほどデータの値は平均値に集まっていることを表す。  
　各データから平均までの距離（**偏差**と呼ばれる）をそのまま平均しても 0 になってしまうので 2 乗している。

**母標準偏差**:

$$
\sigma = \sqrt{\sigma^2}
$$

　標準偏差は分散の平方根であり、データが平均からどの程度離れているかを示している。

**標本分散**:

$$
s^2 = \dfrac{(x_1 - \overline{x})^2 + (x_2 - \overline{x})^2 + \cdots + (x_n - \overline{x})^2}{n - 1}
$$

　標本分散ではデータ数 $n$ ではなく $n-1$ で割る。これは、母平均と違って標本平均 $\overline{x}$ は 標本 $x$ の取り方によって偏りが生じるため。

In [ ]:
# n-1 で割る理由を知りたいなら、次の動画が参考になる
from IPython.display import YouTubeVideo
YouTubeVideo('Ej8TS4a00VM?si=Ldh8YXPBx7OlHGQ_', width=640, height=360)

**標本標準偏差**:

$$
s = \sqrt{s^2}
$$

標準ライブラリの `statistics` モジュールは、統計計算のための関数を提供する。以下は、その主な関数であるが、引数 `data` は、`int`, `float`, `Decimal` または `Fraction` を含むシーケンスである。`data` が空の場合、 `statistics.StatisticsError` 例外が発生する。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `statistics.mean(data)` | `data` の算術平均を返す | 数値 |
| `statistics.fmean(data, weights=None)` | `data` を `float` に変換し、算術平均を返す。`mean()` 関数よりも高速に実行され、常に `float` を返す。`data` の各<br />要素に重み付ける数値のリストを `weights` に指定できる | `float` |
| `statistics.geometric_mean(data)` | `data` を `float` に変換し、幾何平均を返す | `float` |
| `statistics.harmonic_mean(data, weights=None)` | `data` を `float` に変換し、調和平均を返す。`data` の各要素に重み付ける数値のリストを `weights` に指定できる | 数値 |
| `statistics.median(data)` | `data` の中央値を返す | 数値 |
| `statistics.mode(data)` | `data` の最頻値を返す | 数値 |
| `statistics.quantiles(data, *, n=4, method='exclusive')` | `data` の `n` 分位数である `n-1` 個の数値のリストを返す | `list` |
| `statistics.pvariance(data, mu=None)` | `data` の母分散を返す。既に `data` の平均値を計算している場合、それを第 2 引数 `mu` に渡して再計算を省略で<br />きる | `float` |
| `statistics.pstdev(data, mu=None)` | `data` の母標準偏差を返す。第 2 引数 `mu` の意味は `pvariance()` と同じ | `float` |
| `statistics.variance(data, xbar=None)` | `data` の標本分散を返す。既に `data` の平均値を計算している場合、それを第 2 引数 `xbar` に渡して再計算を省<br />略できる | `float` |
| `statistics.stdev(data, xbar=None)` | `data` の標本標準偏差を返す。第 2 引数 `xbar` の意味は `variance()` と同じ | `float` |

In [ ]:
import math
import statistics

# 生徒10人の得点
x = [70, 50, 80, 60, 50, 60, 70, 60, 90, 40]

# 最頻値
assert statistics.mode(x) == 60

# 四分位数
assert statistics.quantiles(x) == [50.0, 60.0, 72.5]

# 中央値
assert statistics.median(x) == 60.0

# 母平均
mu1 = statistics.fmean(x)
mu2 = sum(x) / len(x)
assert mu1 == mu2 == 63.0

# 標本平均 -- 標本は x[2:] とした
sample = x[2:]
barx1 = statistics.fmean(sample)
barx2 = sum(sample) / len(sample)
assert barx1 == barx2 == 63.75

# 母分散
sigma_square1 = statistics.pvariance(x, mu1)
sigma_square2 = sum([(xx - mu2) ** 2 for xx in x]) / len(x)
assert sigma_square1 == sigma_square2 == 201.0

# 母標準偏差
assert round(statistics.pstdev(x, mu1), 1) == round(math.sqrt(sigma_square2), 1) == 14.2

# 標本分散
s_square1 = statistics.variance(sample, barx1)
s_square2 = sum([(xx - barx2) ** 2 for xx in sample]) / (len(sample) - 1)
assert round(s_square1, 1) == round(s_square2, 1) == 255.4

# 標本標準偏差
s1 = statistics.stdev(sample, barx1)
s2 = math.sqrt(s_square2)
assert round(s1, 1) == round(s2, 1) == 16.0

# ある企業の年間売上高(万円)
x = [50, 75, 60, 72]
# 成長率
growth_rate = [xx / x[i - 1] for i, xx in enumerate(x) if i > 0]
assert growth_rate == [1.5, 0.8, 1.2]
# 年平均成長率（幾何平均）
cagr1 = statistics.geometric_mean(growth_rate)
cagr2 = math.pow(math.prod(growth_rate), 1 / len(growth_rate))
assert round(cagr1, 2) == round(cagr2, 2) == 1.13
# 5年目の売上の予測は 720000 × 1.13 ≒ 813,600円

# 1 km の道の行きの速度(km/h)と帰りの速度(km/h)
x = [60, 30]
# 往復の速度（調和平均） -- 距離は 2 km で、時間は (1/60 + 1/30) h
y1 = statistics.harmonic_mean(x)
y2 = len(x) / sum([1 / xx for xx in x])
assert y1 == y2 == 40.0

擬似乱数
--------

**擬似乱数**（pseudorandom numbers）とは、一見乱数のように規則性のない数の並びのように見えるが、再現性があり真の意味で乱数とは言えない数値（の列）を指す。真にランダムであるとは「計算できない」ことであるから、コンピューターが乱数を「計算する」ことは根本的に矛盾している。このため、コンピューターの「計算」によって得られる不規則的な数値は、必ず擬似乱数である。

擬似乱数列を生成する関数を**擬似乱数列生成器**（pseudorandom number generator; PRNG）という。擬似乱数列生成器は、決まった数式と初期値を持ち、初期値から数式を計算して擬似乱数を生成していく。この初期値を**乱数の種**（random seed）という。乱数の種を変えることで、擬似乱数列生成器が生成する数値の列を変化させることができる。

擬似乱数列生成器は、数値シミュレーションの目的で使われる。<font color="red">擬似乱数は再現性があるので擬似乱数列生成器をセキュリティ目的に使用してはいけない</font>。

標準ライブラリの `random` モジュールは、さまざまな擬似乱数列生成器を提供する。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `random.seed(a=None, version=2)` | 乱数の種を指定する。`a` が省略されるか `None` の場合、現在のシステム時刻が使用される。`a` が `int` の場合、それが直接使われる。<br />`a` が文字列やバイト列の場合、デフォルト（`version=2` の場合）では、`a` が `int` に変換され、そのビットがすべて使用される。`version`<br /> は Python 3.2 以前との互換性のためにあるオプションである | `None` |
| `random.random()` | 0.0 から 1.0 までの範囲でランダムな浮動小数点数を返す | `float` |
| `random.uniform(a, b)` | `a` と `b` の間のランダムな浮動小数点数を返す。`a <= b` の場合は `a + (b-a) * random()` と同じ。この計算の丸め処理のために `b` <br />が範囲に含まれない場合がある。`b < a` でもよく、この場合は 2 つの変数を入れ替えた式が適用される | `float` |
| `random.randrange(stop)`<br />`random.randrange(start, stop[, step])` | `range(start, stop, step)` の要素からランダムに選ばれた要素を返す | `int` |
| `random.randint(a, b)` | `a <= N <= b` であるようなランダムな整数 `N` を返す。`random.randrange(a, b+1)` と同じ | `int` |
| `random.choice(seq)` | 空でないシーケンス `seq` からランダムに要素を返す。`seq` が空のときは、`IndexError` 例外が送出される | `seq` の要素 |
| `random.choices(population, weights=None,`<br />` *, cum_weights=None, k=1)` | `population` から重複ありで選んだ要素からなる大きさ `k` のリストを返す。`population` が空の場合 `IndexError` 例外を送出する。<br />`weights` シーケンスが与えられた場合、相対的な重みに基づいて要素が選ばれる。`cum_weights` シーケンスが与えられた場合、累積<br />的な重みで要素が選ばれる | `list` |
| `random.sample(population, k,`<br />` *, counts=None)` | 母集団のシーケンス `population` から選ばれた `k` 個の要素からなるリストを返す。`population` の各要素につき何個重複して選ぶ<br />ことができるかを整数のリストで `counts` に指定することができる | `list` |
| `random.shuffle(x)` | シーケンス `x` の要素の順序をシャッフルする | `None` |

`random.seed()` 関数を省略した場合、Python の `random` モジュールは システムの現在時刻や OS が提供するエントロピーソース をもとに、自動的に乱数の種を設定する。つまり、実行するたびに異なる乱数が生成される。

`random.seed(固定値)` を設定すると、毎回同じ乱数列が生成される。乱数の再現性が必要な場合は `random.seed()` を設定し、予測不可能な乱数が欲しい場合は省略するとよい。

In [ ]:
import random

random.seed("Python")
r11 = random.randrange(10**6, 10**7)
r12 = random.randrange(10**6, 10**7)
random.seed("Python")
r21 = random.randint(10**6, 10**7 - 1)
r22 = random.randint(10**6, 10**7 - 1)
# 乱数の種が同じなら、同じ疑似乱数が生成されていく
assert r11 == r21 == 7648955
assert r12 == r22 == 3710201

x = list(range(10))
assert x == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
random.seed(2)  # 乱数の種を固定
assert random.choices(x, k=4) == [9, 9, 0, 0]  # 重複あり
assert random.sample(x, 4) == [2, 4, 8, 7]  # 重複なし
assert random.sample(x, 4, counts=[4] * 10) == [3, 9, 0, 9]  # 各要素 4 個まで重複を許可
random.shuffle(x)
assert x == [0, 1, 3, 9, 4, 5, 7, 8, 6, 2]  # x 自体が変更される